In [11]:
conversation = """
说话人1 :开始吧，开始吧，我是说话人一我叫王满红，今年28岁。
说话人2 :我是说话人二我是孙玉涛。
说话人3 :我是三号，我是三号。关于一号的发言，我表示质疑。
说话人4 :我是说话人四号，我是演员四号，我非常赞同一号的说话呀。
说话人1 :桌上为什么这么多吃的还有小面包从哪里搞的？
说话人2 :哦，这个都已经注册上了。
说话人4 :我们看一下没有昨天晚上我没注册上我的四号没注册。
说话人4 :上四号刚刚注册。
说话人4 :哪里呀？我只是显示了这个，但是说话人还没还没有显示四号，你看发言人只有三。
说话人4 :还没蹦完。
说话人4 :还没碰完。
说话人4 :你就是那就，那就。那就可以自由聊会儿聊。
说话人2 :但是其实真实的开会的场景，呃，这种重叠讲话的人多吗？其实感觉。
说话人1 :多，他就是一个人还没讲完另外一个人就接着上了。
说话人2 :是的，那在这种情况好像大家表现都。
说话人4 :还有这这些情况下不会让你前面有一个人来讲一句啊，对吧？人有贼多，我靠你那万一两个地域不是相同的，他们前面坐那个做声纹他们，他们就提出这个问题就是如果低于相同的两个人去做声纹，就
是它那个，我觉得80%的那个唤醒错误率了，我都不。
说话人2 :敢，你的意思就是同一个地域的。
说话人4 :他跟一些，那个，那个。
说话人4 :就像龙性别的他那个真的是一模一样的那个子孙也贼哦，不是那个幻影错误率贼高。他说，虽然他搞声纹注册了，但是他们当时承诺的指标是当时上线的指标是就是就是30%嘛，就是唤醒这个误闯率30%
，所以这边还是挺。
说话人4 :高的了。
说话人4 :就说说明他辨识度不高是吧？
说话人4 :呃，对双方辩手都不是很高，就是因为他说那些，嗯，他也说了，华为那些苹果的话都是我们都不行，都不敢，他们好像自己都。
说话人4 :测过的都是都是都不。
说话人4 :守住了，不错。
说话人4 :双方现在最大的问题好像前面。
说话人1 :就是一些鲁网行动。其实如果在他们使用的过程中，就是刚开始的时候提示就是一开始多注册一点长的，我操这***救护车吗？不太好干哦。
说话人1 :是你是你是四号是吧？对啊，这个一直刚才那个厂出来都是在三号。
说话人1 :下面三号才三号。
说话人4 :我不是四号三号。
说话人1 :四号都很模糊。
说话人1 :我是几号？我是几号？
说话人1 :我生日几号？
说话人4 :现在我一直在说话你现在可以分出来这个人是谁吗？
说话人1 :叶山浩，他也是三号。
说话人4 :是吗？我也是三号嘛我是几号？
说话人4 :最终就被你合并了，嗯？
说话人4 :不是不是，不是，是不是？
说话人4 :找找个女生来看一下女生和男生之间。
说话人4 :会不会和平？我靠。
说话人1 :女生区分度高一。
说话人4 :点，女生就会高一点。嗯，异性之间可能区分度高一点。
说话人4 :同性之间，可能这个有点。
说话人3 :现在还有四号吗？一直都是三号的声音，现在是两个人分不出来的。
说话人4 :她也估计也就是类似于生活比较相似度，它结果把三号四号的生物给他提取在一起了，他觉得跟三号也挺近。
说话人4 :的跟四号也挺近，可能三号四号从那个距离图上他。
说话人4 :就靠得很。
说话人2 :近，所以说句后面他就混沌了。
说话人2 :估计是了。
说话人4 :后面后面反正他后面你比较好也是比较上蜘蛛。
说话人4 :他觉得就是明星就行。
说话人1 :他已经初六了，他已经出六了。
说话人2 :运费那就这样子呗，我们到到时候对的逻辑就是说这个时时呢就是容易容易不太准对吧？那就到时候这样子，我明天到时候明天这个逻辑跟她讲，第一个就这个指标，你看今天来不来得及整袋来得及拯
救来得及整来不及整，到时候给他们看一下我们我们那个就是说那个实验室里想的，你就说大家都相当，你不要说谁好。
说话人2 :奇怪，也就说大家其实都差不多，但实际的你就看牛似的，你说这个讯飞的也也这个样子。然后我们也下个结论是比他好还是比他差，还是他有什么优点，他有什么缺点，就你那边多多写一些对吧，然
后那个东西那个那个那个那个，因为你说这个时代这个指标。
说话人2 :统计起来比较比较比较比较难统计哈，都是不同，但是从那个从从。
说话人2 :不管的感受来看是谁的更好对吧。然后再把那个卡片还是做了一个那个离线的吗？那离线的是不是这个结果可以拿过去一回一回放就是拿过去一一上传不就好了嘛，对不对？就拿这个手机上的对不对，
这样子一对比打不进去啊。讯飞讯飞听见讯飞听见打不进去音频那倒不进去，因为你就现在哪个手机开着的。
说话人2 :那不就好了吗？那你你必须要重弄一遍话，大哥，我**咋想的嘞？我操。
说话人2 :红歌，所以就是说这个逻辑这个。
说话人4 :这个也没导出来，应该是个完整的页面。
说话人2 :不应该是每个人的样子，就让它有个预期就做完了。这个东西虽然现在是不是特别稳定，但是现在业界就是这个样子，但我们态度上我们是很好，我们肯定会配合你去优化那些东西，哪些问题我们也总
结出来，对吧？就就就就这个样子啊，嗯，就行了呀。
说话人2 :嗯，好。
说话人2 :你。你明白了吗？就这个。
说话人2 :这个逻辑是可以的。
说话人2 :嗯，应该可以到吧，你可以随便。
说话人2 :就互操。
说话人1 :可以的，可以的，我们测一下短剧的效果。
说话人3 :那个合并的逻辑是。
说话人3 :怎么搞你，你也来一句。
说话人5 :何必那么过去就就是。
说话人4 :如果我们俩人说话非常相近的话他不是就两个人你说话马上说的话他会合并把它。
说话人4 :拽出来成一个人，对吗？
说话人1 :有没有好吃的呀？
说话人1 :这都是些什么零食啊，都是一些垃圾零食。
说话人2 :我还是二号嘛，我票都没有。
说话人2 :啊，你不要跑过来。
说话人2 :讲没必要没事儿这***不正常吗？
说话人1 :开个会？对啊正常一一米多的距离吗？
说话人3 :我到现在不知道我是三号还是四号，你给一个准确的结论吧。
说话人3 :我变成变成五号了。
说话人1 :因为你在吃肉。
说话人1 :吃东西讯飞给的你是六。
说话人3 :嗯，现在东西吃完了啊能回归到四号身份了吗？还是。
说话人4 :应该应该回归不了了，我觉得。
说话人3 :嗯四号四号了嗯？
说话人3 :哎那。
说话人2 :比如说像这样，我们收敛一点就是这个东西能能能能支撑刚刚我说的那个事情，就比如有一个直观的这个那个主观的那个那个比较，所以会好一点，或者一个主观的指标弄出来，或者就结结个结节一段
时间，呃，今天下午让他们陪你去标一下，还是怎么说可以啊。
说话人2 :约十分钟，约十分钟。嗯，能得到这个结论，其实就是说如果能得到结论是这个跟这个是相当的，然后这个是比那个经验里离线的倒进去的要差一些，基本上就就能说得通了吗？逻辑上就可以对的上可
以可以。
说话人6 :是吧？这个感觉是不是没有之前那个靠谱了，最近感觉效果变差了，感觉也可能是不稳定。
说话人3 :幸亏这个还会更新吗？他那个联网。
说话人4 :就你们这个应该更新会。
说话人1 :更新吧，那谁知道呢？他更不更新，我们也不知道。
说话人1 :好，我吃东西的时候他也知道我是。
说话人1 :一级还可以，其实。
说话人2 :对吧？那我们就在群里说一下这个东西已经更新了，让他们在体验体验，就是说明天再对一下，你把这个问题再整整咱这个东西说实话，妈的能做的也就那个样子，那个东西我再回去把那个发布会的讯
飞的那个东西再研究研究。因为昨天王总就问了，你把这个结果等等，就按照那个思路，就把那些问题再以以他们不太懂。
说话人2 :这个专业能听得懂的维度，来来来，讲一讲懂的意思了吗？好吧，嗯，好吧，好吧，你有哪些问题，然后多弄一些，就表明我们这个东西还是经常用，用了很多的那个东西太少了。好吧，嗯。
说话人1 :你刚才那段话我需要。
说话人4 :剪掉是感冒了吗？我。
说话人2 :感觉是感冒了呀。
说话人2 :我**这二七了呀。
说话人2 :带冰袋冰袋。
说话人3 :上回我昨晚还在战斗呢。
说话人4 :你错了还跟我说你回家有事儿结果回家呆着。
说话人6 :真他妈是已经是那都11点了11点了，大哥。
说话人4 :**你这有事儿。
说话人3 :11点干到。
说话人3 :凌晨12点半，一点半要12点半。
说话人2 :加入你这一步，这一步打了。
说话人3 :都干干干干不动了。
说话人3 :昨天我们不也在。
说话人3 :打一会儿回来一看，你们已经开了。
说话人4 :你们晚上几个哪几个？
说话人2 :可热闹了，前天下午要不把你拉。
说话人4 :进去，哇，那个手游我是一打出汗。
说话人2 :我江苏给他介绍一下我们的盛况好吧，明天晚上的。
说话人3 :夏博士刀塔战神。
说话人4 :我怎么都不打Dota我打断我打断由我不。
说话人4 :打手游手游。
说话人4 :说什么出汗？
说话人7 :不行，消费者对1607这边待会要开交流会四点。
说话人2 :四点是吧，我看现在几点好，我们马上就结束了想为什么这么多吃的吃的。
说话人2 :糖豆，你也要参加。
说话人2 :我要参加。
说话人2 :啥？我说啥会。
说话人7 :呀，应届生交流。
说话人2 :我操这还要交流啥喂？
说话人1 :好好好好把垃圾整理一下吧，兄弟们。
说话人4 :哦，怪不得我。
说话人2 :待会可以拿过来让他们测一下你把这个先倒过去四点的时候。
说话人2 :就拿过来。
说话人2 :测，看看交流会我们这个情况，因为。
说话人2 :好多人，那大方没关系我们就极限测试啊。
说话人1 :那一会你拿来呗啊。
说话人2 :我拿过来我就在这儿好不好那你不是要。
说话人2 :参加吗？我不是说没人。
说话人2 :通知我要参加，我不知道没人通知我。
说话人2 :不是发邮件了吗？
说话人2 :9点51好的，你先把它拿过去抖一下，让标注原先去标，然后我们。
说话人2 :继续再弄吧。
说话人2 :OKOK那那个。那个讯飞听见。
"""

gpt_summary = """
会议主题：声纹识别系统优化讨论会

与会人：

王满红（说话人一）
孙玉涛（说话人二）
三号（未确认身份）
演员四号（未确认身份）
叶山浩（未确认身份）
会议议程：

讨论声纹识别系统的性能问题，特别是重叠讲话时的效果。
分析声纹识别系统在同一地域和不同地域之间的辨识度。
探讨声纹识别系统对于男性和女性之间的区分度。
讨论声纹识别系统的更新频率和稳定性。
会议结论：

声纹识别系统在重叠讲话时效果不佳，需要优化处理。
同一地域的声纹识别区分度较低，需要进一步提高。
异性之间的声纹识别区分度较高，同性之间可能存在问题。
更新声纹识别系统是必要的，但需注意稳定性和更新频率。
下一步行动：

对声纹识别系统进行优化，特别是在重叠讲话场景下的处理。
进一步研究提高同一地域的声纹识别区分度。
继续探索异性和同性之间的声纹识别区分度，并进行数据收集与分析。
优化更新机制，确保声纹识别系统的稳定性和性能。
备注：会议中出现了一些未确认身份的说话人，需要进一步确认和记录。
"""

In [12]:
# WxYiyan langchain API test

from datasets import load_dataset
import json
import requests
from collections import defaultdict

from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from inference_api import YiyanInferenceApi, HfInferenceApi
import regex

from langchain.llms import Tongyi
from wenxinyiyan import WxYiyan

def kwargs_test(**kwargs) :
    for key, val in kwargs.items() :
        print(f"{key}:{val}")
    print(kwargs['prompt'])

kwargs_test(prompt='Hello', temperature=0.2, top_p=0.9)

llm = WxYiyan(model_name = 'ERNIE-Bot')
llm('tell me a joke!')


prompt:Hello
temperature:0.2
top_p:0.9
Hello


'当然，这里有一个小笑话：\n\n一只乌龟和一只兔子在森林里比赛跑步。\n\n乌龟说：“你知道吗，兔子？我曾经从太平洋一直走到大西洋。”\n\n兔子回答：“那还不容易，我也会。”\n\n乌龟惊讶地说：“真的吗？那你走的是哪条路？”\n\n兔子回答：“当然是从太平洋银行到亚特兰大。”'

In [13]:
# system role testing

eb_llm = WxYiyan(model_name = "ERNIE-Bot", system="你是OPPO助手，请热情准确地回答用户的问题。")
print(eb_llm("Hello!",  temperature = 0.1, top_p = 0.9, penalty_score = 1.0))

eb_turbo_llm = WxYiyan(model_name = "ERNIE-Bot-turbo", system="你是OPPO助手，请热情准确地回答用户的问题。")
print(eb_turbo_llm("who are you?",  temperature = 0.1, top_p = 0.9, penalty_score = 1.0))

您好！欢迎来到OPPO助手的世界。我是您的智能助手，随时准备为您提供帮助和支持。请告诉我您需要了解什么信息，我会尽我所能为您提供准确和详细的解答。
我是OPPO助手，可以为您提供快捷的智能语音服务。如果您有任何问题或需求，欢迎随时与我联系。


In [14]:
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from wenxinyiyan import WxYiyan

llm = WxYiyan(moddel_name = "ERNIE-Bot", debug=False)

# summerization template
format_template = """
你是一位认真仔细的会议助手，根据以下会议讨论中不同用户发言的内容，请重点总结提取对话核心内容，包括会议主题、会议议程、会议结论以及下一步行动。对话内容如下：\n
{conversation} \n\n
总结的核心内容如下：\n
"""

dialogs = conversation.split("\n")

conversations = []
conversation_chunk = []

chunk_len = 0
CONTEXT_LEN = 1800

for diag in dialogs :
    diag = ':'.join(diag.split(':')[1:])
    chunk_len += len(diag)
    if chunk_len < CONTEXT_LEN :
        conversation_chunk.append(diag)
    else :
        conversations.append("\n".join(conversation_chunk))
        conversation_chunk = []
        conversation_chunk.append(diag)
        chunk_len = len(diag)

if len(conversation_chunk) > 0: 
    conversations.append("\n".join(conversation_chunk))

print(f"len_dialog={len(dialogs)}, len_chunks={len(conversations)}")
# print(f"{len(conversations[0])}, {len(conversations[1])}")

summerizations = []

for (i, c) in enumerate(conversations) :
    example_prompt = PromptTemplate.from_template(format_template)
    grounded_prompt = example_prompt.format(conversation = c)
    completion = llm(grounded_prompt)
    summerizations.append(f"{completion}")
    print(f"###部分纪要-{i}")
    print(f"{completion}")

# overall summerization template
format_template = """
你是一位认真仔细的会议助手，请根据如下几个分段的会议核心内容，总结一份完整的会议纪要，包括会议主题、会议议程、会议结论以及下一步行动。
要求会议议程不要超过10条，会议结论用一句话简单概括。分段会议核心内容如下：\n\n
{summerizations} \n\n
最终总结的会议纪要的如下：\n\n
"""

example_prompt = PromptTemplate.from_template(format_template)
grounded_prompt = example_prompt.format(summerizations = "\n\n".join(summerizations))
# print(grounded_prompt)
completion = llm(grounded_prompt)

print(f"###汇总纪要")
print(f"{completion}")



len_dialog=149, len_chunks=2
###部分纪要-0
会议讨论了关于语音识别和声纹识别技术在会议场景中的应用情况。存在多个人同时发言和声音相似的情况下，语音识别和声纹识别的准确率都会下降。在某些情况下，不同人的声音区分度不够高，容易造成混淆。此外，会议还提到了一些关于指标承诺、测试结果、离线数据等问题，但未得出具体结论。针对这些问题，会议建议配合优化和改善相关技术。
###部分纪要-1
会议主题：讨论一个能支撑所述事情的指标，比较离线和在线倒数据的性能，以及讨论交流会的情况。

会议议程：

1. 讨论指标性能。
2. 体验更新后的产品。
3. 在群里通知更新后的产品体验。
4. 讨论手游和其他活动。
5. 通知参加交流会。
6. 整理垃圾。

会议结论：

1. 应该不能回归。
2. 效果可能不稳定。
3. 产品会更新。
4. 群里的反应不积极。
5. 出汗可能是感冒了。
6. 讨论手游和其他活动很热闹。
7. 参加交流会的人不多。
8. 需要整理垃圾。

下一步行动：

1. 拿过来测试极限情况。
2. 继续弄标注。
3. 在群里通知更新后的产品体验。
###汇总纪要
会议讨论了语音识别和声纹识别技术在会议场景中的准确率问题，建议配合优化和改善相关技术。会议主题为讨论指标性能、体验更新后的产品、讨论手游和其他活动以及整理垃圾等议题。会议结论包括不能回归、效果可能不稳定、产品会更新、群里反应不积极、出汗可能是感冒了、参加交流会的人不多以及需要整理垃圾等。下一步行动包括拿过来测试极限情况、继续弄标注以及在群里通知更新后的产品体验等。


In [16]:
conversation = """
说话人1 :嗯嗯，正是这些没没没没看完啊，就是说，所以我们在细粒度的看一下过来吗？
说话人1 :看一下。
说话人1 :我们每一个每一个指令都要。
说话人2 :单独就是看一下，就是说如果这个要特殊处理的，我们把它标出来标标记出来怎么怎么大概要要怎么怎么怎么处理，还是用通用同样的N元模型换一些语料就可以了，就是说我们去挨个看一下，就这个这
个里面已经合并了。
说话人3 :他这种各个垂类是是单独测嘛，标黄的就行。哎，蓟县的时候有标黄的时候我们统一个测一个，他是这样子的，他测试测他肯定是分这个系的纯预测。
说话人2 :但是当时对的就是说只要平均达标就行了。但是标黄的重点呢，是要达标，像那些不标黄。
说话人2 :的那些东西？
说话人2 :啊，就是即使不达标，最终平均达标也行，大概是这个这个这个逻辑就是。所以说重点看一下那个标房的那些。
说话人3 :那系统工具和系统设置。
说话人2 :这其实没多，没有太大的区别啊，就是他们。
说话人3 :其实感觉咱合到一块。
说话人2 :就行，对对S来讲是可以合到一块。
说话人2 :嗯，可以，那我就把它先去掉就这些，上次上次随便标了一些，先弄一下，就是说这里面可能要讨论的是这个，这个东西就这些，这些是哎。
说话人3 :嗯，就这些。
说话人2 :系统设置应用控制对吧？应用控制，这上面写了一些语言模型的声学模型的，就这些是不是都用顺序，都是在声学模型里面，像比如后，嗯，后面要可扩展性，我们比如要不要置一些那个就假设啊，它
有一个新的配置项目了。
说话人2 :对不对？我们要不要我感觉是不是得支持到那种力度，就我们那个里面那个眼睛里面也有一个有一个相当于列表的什么东西，就是有新的来，至少我们不要每一次新的来，都要去动那个数学模型跟语言
模型有一些可能加到那个里面去，加到清单里面，它也能生生效，然后再结合一些句式。
说话人2 :对对对，系统级的有需要需要还加个列表门系统级都是相对比较固定的。
说话人2 :都比较，呃。
说话人2 :对，就是它每一个版本，比如它是相对固定的，但是可能后面就比如再来就是假设这个引擎我们7.0搞，我们把那些东西都搞了一遍，然后呢8.0，它有做一些特性，9.0有做一些特性，就是说就是只是只
是。
说话人2 :考虑一下这种就比如要不要放一些这这种里面去。
说话人2 :就是弄过这个机制出来还是这种，只要训到声学模型里面去就就可以了。
说话人1 :嗯，嗯，我，我感觉这种的话数学跟语言就就可以。
说话人1 :也对，没没必要单独弄一个。
说话人1 :列表项那个NFC的那那种。
说话人1 :去去搞。
说话人2 :吧哦，那这种真的就是那那就相当于那这种方案的话是不是就是主要还是要把这些数据文本文本弄过来，就是音频文本是搞语言模型的音频。
说话人1 :哎，我想问就是我我们现在那个手机助手里面那个。
说话人1 :线上的数据我们有吗？就是他，他列了这些类，他每一类他都有对应的，比如说视力啊，或者是用户的一些一些。
说话人3 :人家是根据意图统计出来。
说话人2 :它是这个是。
说话人1 :有意图呢？没有音频。
说话人2 :肯定没有，没有音频音频等我们等我们去贝塔以后的。
说话人2 :这个音频了，嗯？
说话人3 :嗯，要不然就是让你读文本。
说话人2 :还是那，那就有文本了，那就把文本拿来拿来。文本是有的文本，王凯，他们那边应该有这个文本，应该是一起一起去搞，因为王凯他们做N意图的那个东西也有，那我们是不是就第一呃，那那一开始呢
，那就把文本拿过来，然后先先去付什么呗，那就音频，然后文本拿。
说话人1 :拿去做个医院吧。
说话人2 :那这种都是不是都变成一类了？
说话人2 :然后最终。那就是。
说话人1 :就是后续，后续的音频我们都能拿到是。
说话人2 :后续能拿到或许能拿到回流啊。
说话人2 :你不还得标注吗？
说话人3 :就是数学，这个时间周期肯定很长。
说话人2 :对啊，后续可以这样子嘛就是说。
说话人2 :嗯，识别的结果跟着意图都已经回流回来了吗？回来回来了以后可以基于那个那个线上的平台去分析吗？有哪些语料好的效果不好的可以倒进去吗？这后面你肯定没什么好的办法，肯定要结合一些工具
加人工的方法去。
说话人2 :去搞这个东西。
说话人4 :我觉得可以把你那个文本拿来然后用我们词语之前搞的那个工具去采集一些像这种系统类比较暴力的词。
说话人2 :有必要去搞一点。
说话人2 :那几天是搞一部分嘛，对不对？
说话人2 :就是说那个要不要人工采？因为人工采的话，也可以找一些找一些人采一些。
说话人3 :对，他们是有那个。
说话人3 :AP APK工具。
说话人2 :吗？对，有工具。
说话人4 :吗？对啊，用工具拆。
说话人4 :就可以了，嗯？
说话人1 :用工具来猜。
说话人3 :之前，之前可鉴可。
说话人4 :说的时候，你看你之前就没录音。
说话人3 :看见两个菜。
说话人1 :最多的是这。这些应用，这些就是。
说话人1 :主要的大类。
说话人4 :高层保证他。
说话人3 :不出错就行，就比如说他这种他。
说话人2 :有个清单，有个APP工具一条一条的显示出来你读，然后它自动保存下来，类似这样子的。
说话人3 :有个两千条三千条给它整理出来，然后。
说话人2 :那也可以弄一下吧，就是把那个技术的效果因为这个专车的，嗯。
说话人2 :你看这个，这个打电话的就不讨论了，打电话就是要带去是的热词你，你现在现在的热词还带带带锯齿。
说话人2 :吗？不带剧时代剧。
说话人3 :是不带锯齿多就是。
说话人2 :哦，那打电话的到时候。
说话人2 :就搞一些高频的，比如打开呼叫什么什么的，那那那些那些打电话，对对对。
说话人2 :你现现在已经预测到那个那个系统里面去了吗？呃，能力是你知道了，但是那个功能的话就是说业务交付的话，呃，我想一想啊，我们对引擎的话，因为现在还自称不同的业务，所以这些热词到时候，
嗯，我们怎么对外开放，是不是就是说远哥我们可以这样就是说。
说话人2 :那他那个通可视化通话掉他一个引擎的类型，对吧？其实说白了就是说我们引擎要知道你是哪个应用掉了，那语音助手掉了，我理解这些东西，语音助手他也不想感知你用什么词句是什么，所以说呃，
就就通过就通过这种方式呗，就是说他调进来的，根据掉进来的应用，或者我们开放一个参数。
说话人2 :就就是什么类型啊，我觉得那个服务的类型啊，这样子就相当于那那是对应语音助手的。那我们这个认识的这些东西，比如我们要预测到里面去的，我们说完的时候就把它都都触发好了，对吧？那嗯。
说话人2 :嗯，那这边再写记记一下。
说话人2 :那这个打电话呢，就大概率只能用就是说要看一下，就是说要带句式的跟梅梅句式的枚举，是的，要不要放枚举？是的，到时候误场那些东西会不会比较厉害，就这个地方要平衡了是吧？对，就是比如
这个。
说话人2 :一般夏普的名字加进去的就是说这个地方。
说话人3 :就无惧色和无惧。是的误闯和待机是的误闯。
说话人4 :带锯是这个场景还有无惧是的吗？
说话人2 :嗯，以前他有那种场景，比如打电话看看你要打给谁王马宏。
说话人4 :那我们也支持你。
说话人2 :以前以前，如果。如果。
说话人2 :有这种技术的话。
说话人1 :那是不是可以给他组组合成再就是的。
说话人1 :对你前面已经有了打电话然后你打电话给后面你，你说个名字是不是，嗯。
说话人2 :这种误闯也太多了。
说话人2 :嗯，对，就是说这里面会有一些，所以以前在在老东家，他们当时做了一个就是叫两极的那个方案，就只带狙，是的，相当于把那个分丢得猛一点，就说的土一点，不带车。是的，我也揪一些，但是我
不能揪太猛。
说话人2 :就是我稍微久一些，所以这里面其实是有一个有一个权衡的，就按照以前的经验，因为因为呃就是搞成，如果搞成完全卡去世的话，因为有些人到时候他说的我们，我们不可能把那些东西都穷举，而且
有的人会觉得或者他他通用里面也会有一些跟年轻人相关的一些东西。
说话人2 :就是说那个东西是没法覆盖，所以二位这个地方可能下来再做一些实验看一下其实。
说话人3 :嗯，那就是我们的食物好。
说话人2 :然后引擎这块到时候跟源哥一起设计一下，就是说怎么样就把它隔离开来，就是说。
说话人2 :然后第三个就是。
说话人2 :这个智慧生活对我们来讲应该没啥的，我就不管他。
说话人2 :了也智慧生活，这智能家居的智能家居也要他用的云呢也是也要有一个词的热词的，他也是有热词。
说话人3 :的智能家具？
说话人2 :它主要是一些设备名设备名，所以这个。
说话人1 :这个设备名是不是就有多用户起的？
说话人2 :比如说你自己用户，企业应该拿到那这个技术要确认一下，呃，拿到怎么拿这个跟张良确认一下啊。
说话人2 :这是端的云云的就要纠错，到时候跟反正二比零整体段落来负责一下云云的那些。如果要成成果啊，他们弄的话，那也要跟他们一起，就整体都都对一下，我觉得里面可能是整体跟你过一下，就是说到
时候每一个细节有什么疑问，到时候再拉我一起讨论。
说话人2 :嗯，匀称。
说话人3 :对，需要纠错。
说话人3 :分成果。
说话人4 :之之前不是说要云澈讨论那个热水方案嘛，要他们去。
说话人2 :支持，对，就是这个再再确认一下日程日程。上次这个笔记没啥的笔记日程呢，那就重点弄一些测试题，把时间呢测一下。
说话人2 :就是这个时间呢，呃，弄一些那个东西应该当时测试已经买了有时间的那个女人。对，就是到时候看一下有没有一些典型错误，这里主要是那个文本规划，对对对，看一下文本的正则，还有一些声学也
可能也会有问题，这个反正还好弄，就是当年好像bug解掉就行了。
说话人2 :都是二维，你再分一下就是这些哪一些谁负责我，我现在就不拍了，个这个就没啥好说的。问答百科搜索闲聊，这个就是语言模型语言模型呢，加速你再看一下这一块，嗯，你的输入要什么？
说话人2 :东西或者你的测试集怎么怎么构造？
说话人3 :我们给你那边在总理这些这些兄弟们。
说话人2 :对，或者就是说你需不需要一些典型的那个那些说法，什么东西啊，那些东西，王凯那边应该也有一些，他们之前带过来应该带过来一些。
说话人3 :嗯，我那边。
说话人2 :有一些，嗯，好，这这反正这个就是家族你负责那下面可能重点要讨论的就是。
说话人2 :音乐视频有声这个对我们来讲属于一类对吧？音乐视频有声。
说话人2 :这个。这个视频是什么？就是电视剧啊呃，就比如播放大宅门第几集就类似播放大宅门这种。其实就是说音乐的话，那其实就是一些嗯高频的歌手名，嗯，想一想啊，就是歌手名。
说话人2 :零，然后还有音乐。
说话人2 :音乐名对吧？然后这个视频的话那其实就是一些热度比较高的。
说话人2 :呃，电视剧。
说话人2 :电视剧，然后。
说话人2 :电影主要就这些，有些人的话其实就是一些那个有声读物的那个，其实跟音乐应该归为类似，类似的那个他主要是相当于一些西方，当然里面的一些东西其实跟声音跟音乐应该隔的，不是不算特别开。
说话人2 :就是，所以这个这几个要重点看一下，它是它是放到通用的，我感觉搞生意的肯定不太现实了，因为这这东西都是海量的。
说话人2 :嗯哦。
说话人2 :像这种。
说话人2 :我们看一下，尽量。
说话人5 :到时候我去一个比如说最近很火爆的一个电视剧出来，名字比较3P，嗯，然后我们本地的那个热词表也更新不了，是比较差，那这种。
说话人4 :怎么开就修复。
说话人2 :啊。所以就是说我们这里面是不是要预置预置两种能力，一种就是比如一有一个句式加一个实体的，对吧？
说话人2 :然后第二个就是说有一些全局的是不是也要放一些放放，但是全局灯可能放少量的，就是跟技术无关的那些那个热词的。
说话人2 :然后马鸿，你刚才讲的其实是一个更新的能力，更新的能力，这个员工下来我们带他们注意一下，后续这个东西是用他们现在好像有两种，一种是拆迁话，升级之前不是弄弄那个吗？可以去推模型推那
个东西，还有后面还有个什么应用，什么应用更新器啊，这个时候下面我们给大家说一下，或者说语音助手怎么怎么去升级，弄那些东西可以。
说话人2 :所以那。那那个工程呢，我们就先不讨论，我们就先讨论怎样能把的解决翠玉准确率做上去就就说白了。
说话人5 :这个剂量，你那。
说话人5 :个能支持这么大规模？
说话人5 :我感觉还挺难的哎。
说话人1 :Top，比如说top五千。
说话人1 :如果每一类都有都有那么多的话，可能有点。
说话人4 :美丽，至少透透不如两三千是有点。
说话人2 :嗯，那个你指的是声学模型吗还是什么？
说话人3 :我也是真心的就是那边就单车这种事。
说话人2 :怎么这种事就是用一个FFFFC做的就带一些句式他会。
说话人5 :多大的规模？
说话人2 :这个很大的，这个他们还都放得很。
说话人2 :大，但是颜色。
说话人2 :的，它主要是原车的，嗯，端侧的也也放了一起，也放了，也放了，也也放了这些东西，其实，呃呃，嗯，不知道谁有没有分界的车，或者就是拿那个华为那个那个手机上，他是他也是搞了一些，我觉
得是不是这个样子，就是说我们根据热度。
说话人2 :先看一下，就是说这个里面大概是什么量级的就加速你那边看一下这个之前它这个实体是，这是这个有声是不是比较少，音乐视频应该相对比较多，或者根据一些加进去。
说话人3 :和那个颜色？
说话人3 :差距还是蛮大的，因为这块比较难。
说话人2 :比较难，嗯，比你现在是加了源文件里面的时候。
说话人5 :对肯定语言模型对他的激励很小。
说话人2 :这种就是相当于现在我们的方案里面就是说语言模型就是一共。
说话人3 :有几百万的，然后还没视频呢，还在找。
说话人2 :这种肯定不能用模具是的，那或者用一些。
说话人2 :带锯？是的，就比如以前老人家的做法，它会有一个，比如播放。
说话人2 :对吧？什么什么？
说话人2 :就加一个槽位对吧？那是个槽位，他们以前都是这样子，就一个槽位，一个一个大鸟服务一个槽位，然后后面这个比如。
说话人2 :播放is single对吧？是一个曹魏然后再加上一个什么什么什么哪个哪个哪个咯，对吧他他，他他，他有有些句式，有些脚本在里面弄，然后那些曹魏他都放了一些文件，就像那个TXT放到里面去，然后
只要更新那个东西就行了，所以那个搞出来就是单句式的那个效果，所以那个其实就是用FS。
说话人2 :太高了，所以我们也看一下我们对N个词里面。
说话人2 :能搞这么大吗？搞不了这么大的时候，那多大呀。比如弄个几10K。
说话人2 :其实其实其实K呃几10K的实体这一。
说话人3 :类，那就万级别。
说话人3 :的，嗯。
说话人1 :呃，够呛了。
说话人1 :一个音乐就几十。
说话人1 :K那视频还还几10K。
说话人1 :你视频这个剧是怎么怎么设定？
说话人2 :里，他也是一件类似于播放的。
说话人2 :那些东西的他其实之前也是基于。
说话人2 :那个回流回来的那个数据，他。他那个大数据平台会有一个有一个聚类就分析这个意图下，呃排第一名的句子是什么？排第二名的句子是什么，排第三名的是什么说法，就是类似这样子的那个搞出来的
一些东西。
说话人5 :这个说法还挺复杂。
说话人5 :的播放，某某某的某某歌曲。
说话人5 :两个草原。
说话人2 :对以前专门他们搞找了一些合作方，就专门来写这种句式的。就是说那个里面是一个是一个Python的东西，然后他再来把他把他把那个fst的那个东西再展开，就再再弄，其实比他就是那个东西变成就是
一条一条SI的路径嘛，对不对？嗯。
说话人4 :然后他最后FSA多大？
说话人3 :对，就是说。
说话人2 :名字的打过去了，云测云测的。
说话人2 :所以我们也没有云测的，它是这个样子的，它有个通用的sst，就是那个就跟着我们通用原文，这个世上记得还有一个叫垂垂类的那个。
说话人2 :东西，那个是几十兆？
说话人3 :跟你刚才我们也是一个成熟的。
说话人4 :通用的话，电影和音乐。
说话人2 :之类没有，没有这个不通用就是就相当于我们的。
说话人2 :日常语言模型就是那个以前的一个，它就相当于以前的是CD C++那个fst的那种那种方式嘛，就相当于那CDC前面声学模型后面那个fst是个语言模型。
说话人2 :但是他的FC有通用的fsd还有一个叫垂域的fft垂域的fft就是用人工，他是配的一些规则，加加一些加一些词典啊，那些东西构成的。然后这个通用的FS跟那个跟那个垂泪在做PK，就是说哪哪边的分高，
或者有一些有一些原则里面就是。
说话人2 :用用哪个？
说话人2 :结果就类似这样子。
说话人2 :就以前，大概。
说话人2 :的量，比例的，它，比如说你上面有。
说话人1 :有那么多类每一个类的都要过吗？
说话人2 :他，他，他那个有好多子的他我记我印象没记错了，他应该把车上的八类有有音乐的对吧？有电话的有哪些呢？
说话人2 :我觉得就是错了。
说话人2 :它也是一些高频。
说话人5 :是根据句式来筛选。
说话人2 :对对对，就是它组合成一个大的ffs，其实是平时配的时候是八八种。
说话人6 :就和他显示如果是有剧社的话，那就可以根据技术去去分类。就是每每一类，它会有一个单独的。
说话人5 :但是播放这种说法就是视频和音乐感觉。
说话人2 :那没关系，那就把它合并吧就这种你认为它类似的就合并吗？其实对对对。
说话人2 :可以把把那个槽位里分成两个文件也没关系，那那后面就相当于维护吧。你句式那东西可以搞得一个矛盾都都都没关系。所以其实就是要讨论一下我们这个这个N的这种，这个这肯定是几10K。
说话人3 :的规模，存储的存储的话，这些。
说话人3 :这些有存钱要增大一些这个肯定要存储吗？嗯。
说话人2 :对他，他当时就就就有这个词典，其实就跟就跟PS里面，我们现在不也有很多词典吗？放在里面的。
说话人2 :他你你放个几10K的应该也没多大吧几十。
说话人2 :K的，比如说。
说话人2 :大概就几10K的时点的话应该也没多少吧。中文中文的也东西，因为估计一兆的一兆都可能都没有。
说话人2 :包括上面这个其实刚刚那个没过那么细啊。这个里面其实还有一个就是。
说话人2 :那个应用就是一些APP APP APP的东西，APP的东西是不是最好也要做成，打开关闭卸载有带一些锯齿的你，你你那些东西进去。
说话人4 :这个倒跟那个刚才一一。
说话人2 :那个是一类方案，所以就是说这一类这一类方案可能要看一下就是是怎么个搞法。
说话人2 :就相当于我们现在恩恩的那个里面有带锯似的，也不带锯是，但是规模搞不了这么多，是这个意思是吧？
说话人2 :那它扩展了是有什么扩展了以后是有什么负面的那个那个东西呢？是解码速度，比如说增大之后对吧？对对对，理解，应该是效果不好吧，如果加多了。
说话人1 :变得，但是像那天我后来网上看见那个那个如果它是根据阴气去过滤的话。
说话人6 :我。我觉得问题不是很大。
说话人5 :但是根据因去过滤的话，你得。
说话人4 :把所有的资料。
说话人4 :再搜索一遍你端菜能有这个算力。
说话人6 :嗯，那来来chillen就是。
说话人6 :一样的问题？
说话人6 :等车怎么搞的？
说话人5 :SE，稍微稍微快一点。
说话人5 :这个是完全就是匹配是的。
说话人5 :嗯，具体过去。
说话人5 :你相当于每一次识别过滤。
说话人5 :比如说两万两万多条词表全部都有朋友。
说话人2 :我们里面不也有FS的手机嘛。
说话人2 :内存的东西是怎么容？
说话人3 :得嘞你哪个？
说话人2 :你说我们现在锻造引擎里面呢？
说话人7 :现在呢，就是说我们现在是给其他业务提供的，就相当于他们传文本，我们直接在线构建的。就是说现在如果是我们自己内置的话，我们就相当于自己先构建好之后，然后引擎。
说话人3 :宇宙，宇宙请求的。
说话人2 :时候我们可以。
说话人2 :下载就可以了，我我我我可能没。
说话人2 :想清楚，我记得曾经上次和代码的时候不是说他那边也有和什么FS的那个fst的那个东西。仓库嘛，我就是说我们现在的这个可见，可从那个里面的热词是用的哪一种方式搞的呢？就是SB，就是SB，就是
搞一个FC队是实时构建吗？
说话人7 :就相当于不同业务。我们现在是热食农民。
说话人2 :他是说，嗯，那这个FSA跟那个恩呢恩的那个热词恩的热词现在有人用吗？
说话人7 :没人用是吧？现在能力能力需要开发吗？如果确定这个方案的话，你8.0应该是需要。
说话人5 :我想一下主要的风险就是如果你把所有的表都合成一个的话，那整体体积非常大。
说话人4 :再转车，一个设计到空间。
说话人7 :嗯，除非是那种除非是我们根据祭祀有句式句式出来之后然后去不同的句式我们加载不同的fft。
说话人7 :就不是不是这样的话，就是可能召回会比较。
说话人2 :对，其实比较现实的就是说这些东西你如果不加上去是搞进去，我估计也也也也比较难啊。这个要家具是要要家具是。
说话人2 :所以在大家那大概率是不是就两条路径，一个就是用N的方法做一个就是用a还是用老的。
说话人2 :Fsd的方法做。
说话人2 :最近不是有一种类的类的。
说话人2 :那个语言模型，卡斯贝丝的那个。
说话人2 :什么类主要分的。
说话人7 :细一点，小一点，它每一个图都会。
说话人5 :想我想的也是这个东西，但是用你那。
说话人4 :个东西，最后到最后不也得。
说话人5 :用华为的那个fsd嘛，就会走到。
说话人4 :你那个相当于。
说话人4 :就是一个做一个归类嘛话术前面。
说话人4 :我们出发。
说话人4 :的那个类？
说话人7 :要分的小细一点小一点，它每一个图都会小，这样他时时去运行的时候加载。
说话人3 :起来，就会小一点。
说话人4 :呃，那那那个像电影电视剧都是这种播放类的不都一样。
说话人4 :的嘛对。
说话人7 :像那种比较。
说话人7 :大，这其实你本质上还是iPhone fst对对对，本质上还是fft。
说话人5 :还有一个问题就是如果你更新的话会涉及到fst的更新。
说话人3 :嗯，所以这两个比较大这个之前思远。
说话人5 :远哥，今天不是提高这个东西。
说话人3 :可以看一下他。
说话人3 :有什么意见吗？
说话人2 :哦，刚刚好像也不太公寓。
说话人2 :看一下。
说话人4 :主要是看一下这块空间会要。
说话人5 :会变多大？把整个的配合。
说话人2 :起来，那比如我们目标就先做一个支持实体不超过100K的，我们评估一下要多少，我觉得一百K10万个。
说话人2 :是不是界限的音乐也能先用的是吗？热度特别不行再改吧，就像。
说话人2 :一个出版的。
说话人1 :我，我给你十万块钱端上游过桥啊。
说话人3 :就是内存，内存会不会就你十万个那个用fsd加载到那个内存里面？
说话人2 :还好吧，那个都不会吃太多内存的。
说话人2 :那个东西。
说话人4 :Fst的加上就是。
说话人5 :保护内存，嗯，对，所以我说恩的稍微。
说话人5 :对端侧。
说话人5 :有好一点。
说话人5 :可能效果上血压计量的盐。
说话人5 :试一下。
说话人3 :两条路都走。
说话人4 :这个就是空间跟效果。
说话人7 :来平衡那个N和那个热词计算的时候都是通过那个注意力算的这样它那个搜索的那个不也是很大的一个标也是几万条，一个一个他们先过滤。
说话人4 :就会少很多。
说话人6 :比如说你五千个，它过滤完。
说话人6 :可能就剩十几个。
说话人7 :那个顾虑和这个F。
说话人7 :IT部也可以结合一些。
说话人6 :嗯，也可以。
说话人2 :那这个下来是不是在做一些实验对比言论，其实，嗯。
说话人4 :不太好结合。
说话人5 :过滤那个是。
说话人4 :是一个动态的你fst构图是一个静态的。
说话人7 :这两个如果可以和一起一起搞，一起加点。
说话人2 :这三个都是一起的。
说话人2 :只是三个一一般做法。
说话人3 :有声的话感觉，嗯，其实不是绝对的哈。重点达标球队的话，嗯，就看音乐和视频。
说话人5 :你看，少了一个地址导航。
说话人2 :对吧？还没到呢。
说话人2 :那下面到新闻就不搞新闻，这个也做不了啥了，就通用的了。
说话人2 :嗯，天气，天气，这个，这个，这个。
说话人2 :这个一般的句子就比如什么，什么天气怎么样，什么什么什么东西的，它也是有一些句式的。
说话人2 :但是这个这个规模应该没那么大，应该到县及县级市，应该上次看的是多少个人，我想想啊，一个省大概有十几个地级市，全国三十三百多个地级市，每个地级市下面一个几千个几千个他们。
说话人2 :区区跟现实一样的三级三级行政嘛省市宪法，嗯。
说话人2 :嗯，所以说这应该是几千个量级的。
说话人2 :这个家族你看一下，我看上次测测了一下，好像有一些效果也出不来。
说话人7 :这是哪哪个版本？
说话人2 :就上次就是拿尾。
说话人3 :那个当时应该是应该是七点。
说话人3 :应该是微微17还是V16的吧。
说话人2 :就是。那就从这个从这个业务的特点来看，也看一下是不是放到语言模型里面就可以了，就是加一些说法还是。
说话人2 :还要不要再搞一些那个那个词儿或者什么的那个那个那个这次的方式去做，是不是？
说话人3 :宋词应该不是不能考。
说话人2 :那就这个就原本再看一下效果。
说话人2 :日历，这也是时间的部分，因为这这些都没有了，这些不搞了，这些我们暂时也没那么精力搞这些东西。翻译翻译，这个要好好看一下，翻译比较恶心。翻译它会讲什么，上次讲过了什么Apple的中文。
说话人2 :什么类似类似于这这种，它其实就是有有不少的英文在里面。
说话人2 :那其实就是对我们的挑战，就是说那些常见的英文要识别识别的相对准一些。我们现在英文好像也改，还不太行。
说话人2 :压力给到了。
说话人5 :给到了思域，嗯？
说话人3 :英文，反正没训练数据。
说话人2 :我们之前是不是就没有这种中式英语的一些训练数据，是吧？没有。
说话人5 :只有纯英文的。
说话人4 :然后加上一些中医粉ts的生产能力。
说话人6 :那个开源的那个状态。
说话人6 :不好，未来开源的那个吗？啊，对对对。
说话人5 :那是教育类的然后就统计了就七七二十几个小时。
说话人7 :之前是有搞那个APP收集了。
说话人7 :那个，我们可以把。
说话人7 :人群放大一点嘛，搞成各种包子这样。
说话人5 :把它把它外面去，是吧？
说话人5 :那奖励怎么发放？
说话人5 :一百块。
说话人5 :钱你别。
说话人5 :穿衣服一分钱。
说话人2 :这个也相对少，还是要靠那个数据回流就是等这个白搭测试这个如果短期内没什么，主要是靠数据驱动的话，那就不管了等等那个线上数据回来再再去标再去搞吧。嗯。
说话人5 :这个回流还能到800W。
说话人5 :上山之后。
说话人2 :他们说7.2就要去陪她对，那下面就到了。
说话人2 :地图导航了。
说话人2 :导航这就是POI的那些东西了这是不是也没啥？
说话人2 :这个就只有语言模型里面搞一些搞热度了，其他也没什么章法。
说话人2 :看这些锤与我们能做好的下面这些银行这些都很小的就不管它了。其实重点来看的话，我们能做一些优化的，其实就是上面这一类的，对吧？那智慧生活的。
说话人2 :然后日程代办的，然后还有就这这几个，其实这个这个也要看一下天计算的那个那个也要看一下，计算的也要看一下。
说话人2 :你再重新标个颜色吧这个。
说话人2 :这个我看一下这样子一看看不清了。
说话人8 :现在就是不知道那个恩恩的热词的效果怎么样，就是肯定宇宙场景肯定还得有那个很多那个bequeath斑点的问题。
说话人8 :到时候怎么去修复？
说话人8 :不知道加一个词进去能不能接受。
说话人4 :就比如说他说导航到哪里那个地方不准。
说话人4 :然后你指望语言模型去更新一次？
说话人8 :那个太慢了。
说话人3 :应该给我，那就直接用先用完它自己去加一个兜底的兜底的一个策略，然后恩的，如果实现好的话，再再再加上或者替换。
说话人2 :主要就是这些水域了，大家都看一下是不是就上面这些，我们有数据的对吧？我们搞一搞，然后这个通打电话的，然后这个智慧生活热词的这些这些时间呢？
说话人2 :可以优化，这些没法搞了，这些就是这些，这些属于中等中等规模的那个那个那个中等规模实体的对吧？然后这个天气也是中等规模实体的，然后计算。"
"""

In [17]:
# Yiyan langchain integration demo

import langchain
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain, StuffDocumentsChain, ReduceDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.schema.document import Document
from wenxinyiyan import WxYiyan

# llm = YiyanInferenceApi("ERNIE-Bot", debug=False)
llm = WxYiyan(temperature=0.1, model_name = "ERNIE-Bot")
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")

dialogs = conversation.split("\n")

conversations = []
conversation_chunk = []

chunk_len = 0
CONTEXT_LEN = 1800

for diag in dialogs :
    diag = ':'.join(diag.split(':')[1:])
    chunk_len += len(diag)
    if chunk_len < CONTEXT_LEN :
        conversation_chunk.append(diag)
    else :
        conversations.append(Document(page_content="\n".join(conversation_chunk)))
        conversation_chunk = []
        conversation_chunk.append(diag)
        chunk_len = len(diag)

if len(conversation_chunk) > 0: 
    conversations.append(Document(page_content="\n".join(conversation_chunk)))

print(f"len_dialog={len(dialogs)}, len_chunks={len(conversations)}")
# print(f"{len(conversations[0])}, {len(conversations[1])}")

langchain.debug = False
# summerization template
format_template = """
请根据以下会议讨论中不同用户发言，总结提取对话的核心内容。对话内容如下在 >>> 和 <<< 之间：\n
>>> {conversation} <<<\n
总结的核心内容如下：\n
"""

map_prompt = PromptTemplate.from_template(format_template)
map_chain = LLMChain(llm = llm, prompt = map_prompt, verbose=False) #, output_key="map_summarization")

# overall summerization template
format_template = """
你是一位认真仔细的会议助手，请根据如下>>> 和 <<<之间的几个分段的会议核心内容，总结一份完整的会议纪要，包括会议主题、会议议程、会议结论以及下一步行动。
要求会议议程不要超过10条，会议结论用一句话简单概括。\n\n
>>> {summerizations} <<<\n
最终总结的会议纪要的如下：\n
"""

reduce_prompt = PromptTemplate.from_template(format_template)
reduce_chain = LLMChain(llm = llm, prompt = reduce_prompt, verbose=False) #, output_key="reduce_summarization")

comb_doc_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="summerizations",
    verbose=False

)

reduce_doc_chain = ReduceDocumentsChain(
    combine_documents_chain=comb_doc_chain,
    collapse_documents_chain=comb_doc_chain,
    token_max=4000,
    verbose=False
)

map_red_chain = MapReduceDocumentsChain(
    llm_chain=map_chain, 
    reduce_documents_chain=reduce_doc_chain,
    document_variable_name="conversation",
    return_intermediate_steps=False,
    verbose=False
)

print(f"###汇总纪要")
print(map_red_chain.run(conversations))

# grounded_prompt = example_prompt.format(summerizations = "\n\n".join(summerizations))
# print(grounded_prompt)
# completion = llm(grounded_prompt)

# print(f"###汇总纪要")
# print(f"{completion}")



len_dialog=149, len_chunks=2
###汇总纪要
会议主题：语音识别技术的准确性和区分度

会议议程：

1. 讨论不同语音识别技术在同一地域的准确性和区分度。
2. 讨论声纹注册的问题。
3. 讨论混淆和合并等语音识别技术上的问题。
4. 讨论零食和身份识别的问题。

会议结论：

语音识别技术在同一地域的准确性和区分度不高，华为、苹果等公司的声纹注册技术也存在不足。此外，还存在混淆和合并等技术问题。零食和身份识别等问题也需关注。

下一步行动：

1. 对语音识别技术进行改进和更新，提高准确性和区分度。
2. 进一步研究和开发声纹注册技术。
3. 解决混淆和合并等技术问题。
4. 对零食和身份识别等问题进行研究和探索。
